In [ ]:
# DisulfideBond Playground
# Playing with the DisulfideBond class
# Author: Eric G. Suchanek, PhD.
# (c) 2022 Eric G. Suchanek, PhD., All Rights Reserved
# License: BSD
# Last Modification: 1/16/23
# Cα Cβ Sγ

# important preamble

import pandas as pd

import pyvista as pv
from pyvista import set_plot_theme

from Bio.PDB import *

# for using from the repo we 
import proteusPy
from proteusPy import *
from proteusPy.data import *
from proteusPy.Disulfide import *

# override any default PDB globals
# location for PDB repository
PDB_ROOT = '/Users/egs/PDB/'

# location of cleaned PDB files - these are not stored in the repo
PDB_GOOD = '/Users/egs/PDB/good/'

# location of the compressed Disulfide .pkl files
MODELS = f'{PDB_ROOT}models/'

# pyvista setup for notebooks
pv.set_jupyter_backend('ipyvtklink')
set_plot_theme('document')


In [ ]:
from proteusPy.Disulfide import Disulfide
from proteusPy.DisulfideLoader import DisulfideLoader
from proteusPy.DisulfideList import DisulfideList

PDB_DIR = '/Users/egs/PDB/good/'    

SSlist = DisulfideList([],'ss')
SSlist = load_disulfides_from_id('5rsa', pdb_dir=PDB_DIR, verbose=False)
SSlist.pprint()

In [ ]:
# Comment these out since they take so long.
# Download_Disulfides(pdb_home=PDB_ORIG, model_home=MODELS, reset=False)

#Extract_Disulfides(numb=1000, pdbdir=PDB_GOOD, datadir=MODELS, verbose=False, quiet=False)

PDB_SS = None
#PDB_SS = DisulfideLoader(verbose=True, picklefile=SS_PICKLE_FILE, pickle_dict_file=SS_DICT_PICKLE_FILE,
#                        torsion_file=SS_TORSIONS_FILE)

PDB_SS = DisulfideLoader(verbose=True, subset=False)

ss_list = DisulfideList([], 'tmp')
PDB_SS.TotalDisulfides


In [ ]:
ss0 = PDB_SS[0]
ss1 = PDB_SS[0]
ss2 = PDB_SS[1]
ss3 = PDB_SS[1]
sslist1 = DisulfideList([ss0, ss2], 'tmp')
sslist1.Torsion_RMS()
sslist1.display()

In [ ]:
new = DisulfideList([], 'tmp')
sslist = PDB_SS.getlist()
ssmin, ssmax = sslist.minmax_energy()
ssmin.pprint()
ssmax.pprint()
minmax = DisulfideList([ssmin, ssmax], 'minmax')
minmax.display()


In [13]:
ssmin.pprint()
ssmin

AttributeError: 'Disulfide' object has no attribute 'torsion_length'

In [ ]:
low_list = DisulfideList([],'neighbors')
low_list = ssmin.Torsion_neighbors(sslist,20)
low_list.length()


In [ ]:
ssmin, ssmax = sslist.minmax_distance()
ssmin.pprint()
ssmax.pprint()

minmax = DisulfideList([ssmin, ssmax], 'minmax')
minmax.display()

In [ ]:

# one disulfide from the database
ss = Disulfide()
ss = PDB_SS[0]
#ss.pprint_all()

# get all disulfides for one structure. Make a 
# DisulfideList object to hold it
ss4yys = DisulfideList([], '4yys')
ss4yys = PDB_SS['4yys']

#ss4crn = DisulfideList([], '1crn')
#ss4crn = PDB_SS['1crn']

tot_ss = len(ss4yys) # number off ssbonds
print(f'tot {tot_ss}')


In [ ]:
print(f'{ss4yys.Torsion_RMS()}')

In [ ]:
sslist = PDB_SS.SSList
tors= sslist.torsion_df
tors.head()

In [ ]:
ca_df = sslist.distance_df
ca_df.head(10)

In [ ]:
ss6fuf = PDB_SS['6fuf']


In [ ]:
ss1 = ss4yys.get_by_name('4yys_22A_65A')
ss1

In [ ]:
Check_chains('4yys', PDB_GOOD)

In [ ]:
ss4yys_a = ss4yys.by_chain('A')
ss4yys_a.Torsion_RMS()


In [ ]:
ss4yys_b = ss4yys.by_chain('B')
ss4yys_b.Torsion_RMS()

In [ ]:
ss4yys_a1 = ss4yys_a[0]
# print(ss4yys_a1.repr_ss_coords())


In [ ]:
ss4yys_b1 = ss4yys_b[0]
ss4yys_b1

In [ ]:
ss4yys_a1.Distance_RMS(ss4yys_b1)

In [ ]:
ss4yys_a1.Torsion_RMS(ss4yys_b1)

In [ ]:
ss4yys_a1 == ss4yys_b1

In [ ]:
chns = ss4yys.get_chains()
ss4yys.has_chain('yyy')
chns

In [ ]:
# load SS bonds by PDB ID
ss1 = PDB_SS['4yys']
print(ss1)
print(ss1[0].get_full_id())


In [ ]:
# you can loop over the IDList list and extract by ID
#
for id in PDB_SS.IDList[:5]:    # just show last 5
    # get the SS bonds for the given ID
    ssb = PDB_SS[id]
    numb_ss = len(ssb)
    print(f'ID: {id} has {numb_ss} Disulfides:')
    for bond in ssb:
        print(bond)
    print('\n')
    

In [ ]:
ss_list = PDB_SS.getlist()
ss0 = ss_list[0]

print(ss0.proximal_residue_fullid)
print(ss0.chi3)
len(ss_list)


In [ ]:
# routine creates 2 lists  for left-handed and right-handed disulfides 
ss_list = PDB_SS.getlist()
left_handed = DisulfideList([], 'left_handed')
right_handed = DisulfideList([], 'right_handed')

i = 0

for i in range(len(ss_list)):
    ss = ss_list[i]
    if ss.chi3 < 0:
        left_handed.append(ss)
    else:
        right_handed.append(ss)


print(f'Left Handed: {len(left_handed)}, Right Handed: {len(right_handed)}')



In [ ]:
from proteusPy.Disulfide import Disulfide

# make some empty disulfides
ss1 = Disulfide('ss1')
ss2 = Disulfide('ss2')

# make a DisulfideList containing ss1, named 'tmp'
sslist = DisulfideList([ss1], 'tmp')
sslist.append(ss2)

# load the PDB Disulfide database
PDB_SS = None
PDB_SS = DisulfideLoader(verbose=True, subset=True)

# extract a disulfide with typical index
ss1 = PDB_SS[0]
#print(f'{ss1.pprint_all()}')

# grab a subset via slicing
subset = DisulfideList(PDB_SS[0:10],'subset')

In [ ]:
rms = subset.Torsion_RMS()
rms

In [ ]:
torsions = PDB_SS.getTorsions()
torsions.sort_values(by=['energy'], ascending=False, inplace=True)

torsions.head(10)

In [ ]:
idx_max = int(torsions['energy'].idxmax())
print(f'IDMAX: {idx_max}')
ssmax = PDB_SS[idx_max]
ssmax

In [ ]:
toget = 200

badlist = torsions['ss_id']

bad_SS_list = DisulfideList([],'20 top high energy')
ss = Disulfide()

for i in range(toget):
    ssid = torsions.iloc[i]['ss_id']
    ss = PDB_SS.get_by_name(ssid)
    #print(f'ID: {ss.name}: {ss.energy:.2f} kcal/mol CA: {ss.ca_distance:.2f}')
    bad_SS_list.append(ss)



In [ ]:

tors_bad_rms = bad_SS_list.Torsion_RMS()
tors_bad_rms

In [ ]:
goodlist = torsions['ss_id']

good_SS_list = DisulfideList([],'20 top low energy')
ss = Disulfide()

for i in range(toget):
    if i == 0:
        continue
    
    ssid = torsions.iloc[-i]['ss_id']
    ss = PDB_SS.get_by_name(ssid)
    #print(f'ID: {ss.name}: {ss.energy:.2f} kcal/mol CA: {ss.ca_distance:.2f}')
    good_SS_list.append(ss)


In [ ]:

tors_good_rms = good_SS_list.Torsion_RMS()
tors_good_rms

In [ ]:
from scipy.spatial import distance_matrix

good_array = good_SS_list.get_torsion_array()
bad_array = bad_SS_list.get_torsion_array()
good_array.shape

In [ ]:

dm1 = distance_matrix(good_array, good_array)
dm2 = distance_matrix(bad_array, bad_array)



In [ ]:
import numpy as np

def EDM(A, B):
    p1 = np.sum(A**2, axis=1)[:, np.newaxis]
    p2 = np.sum(B**2, axis=1)
    p3 = -2 * np.dot(A, B.T)
    res = p1 + p2 + p3
    #res = numpy.sqrt(res)
    return res


In [ ]:
dm3 = np.sqrt(dm2)

In [ ]:
import plotly_express as px
fig = px.imshow(dm1)
fig.show()

In [ ]:
import plotly_express as px
fig = px.imshow(dm2)
fig.show()

In [ ]:
dm1.shape
dm1

In [ ]:
df = pd.DataFrame(dm1.copy())
df.describe()

In [ ]:
x = np.arange(0,20, 20)
y = np.arange(0,20, 20)
z = dm1
x, y = np.meshgrid(x, y)


In [ ]:
# Create and plot structured grid
grid = pv.StructuredGrid(x, y, z)
grid.plot()